In [7]:
import os
import json

In [13]:
from datasets import load_dataset 

dataset = load_dataset("multi_news",split="train")

# os.mkdir("./Dataset")

Found cached dataset multi_news (C:/Users/Francisco/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


In [11]:
# dataset.to_json("C:/Users/Francisco/Desktop/SmartNews/nlp_section/Dataset/train.json")

dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [19]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset):
    text_data.append(sample['document'].replace('\n',''))
    text_data.append(sample['summary'].replace('\n',''))
    if len(text_data) == 10_000:
        with open(f'./Dataset/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
with open(f'./Dataset/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

100%|██████████| 44972/44972 [00:04<00:00, 10364.02it/s]


<h1 style="color:red"> <b> Usar para retirar unicode </h1>

In [33]:
data = []
with open("./Dataset/train.json") as file:
    for line in dataset:
            jsonObj = json.loads(line)
            document = jsonObj["document"]
            summary = jsonObj["summary"]
            data.append((document, summary))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [20]:
from pathlib import Path

paths = [str(x) for x in Path('./Dataset/').glob('**/*.txt')]

In [21]:
from tokenizers import BertWordPieceTokenizer

wordPiece = BertWordPieceTokenizer()

wordPiece.train(paths)

In [22]:
#os.mkdir('./tokenizer')

wordPiece.save_model('./tokenizer')

['./tokenizer\\vocab.txt']

In [26]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./tokenizer')

In [7]:
import torch

In [27]:
with open('./Dataset/text_0.txt','r',encoding='utf-8') as fp:
    lines = fp.read().split('\n')


sample = tokenizer(lines,padding='max_length', truncation=True, max_length=512)

In [28]:
labels = torch.tensor([x for x in sample['input_ids']])
mask = torch.tensor([x for x in sample['attention_mask']])

In [29]:
input_ids = labels.detach().clone()
rand = torch.rand(input_ids.shape)
mask_arr = (rand < .15) * (input_ids > 2)
for i in range(input_ids.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    input_ids[i,selection] = 5 #[MASK] token

In [30]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}


In [31]:
dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [32]:
from transformers import BertConfig

config = BertConfig(vocab_size=30000, 
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1)

In [33]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(config)

In [34]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [35]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

c:\Users\Francisco\Desktop\SmartNews\nlp_section\venv\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0:   8%|▊         | 50/625 [12:27<2:23:16, 14.95s/it, loss=4.23]


KeyboardInterrupt: 